# K8S Resource Test


In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## Show ServerConfigurations Available

In [ ]:
!kubectl get serverconfig -n seldon-mesh

In [ ]:
!kubectl get serverconfig -n seldon-mesh mlserver -o jsonpath='{.spec}' | jq .

## Deploy MLServer Instance

In [ ]:
!cat ../../operator/samples/servers/mlserver1.yaml

In [ ]:
!kubectl create -f ../../operator/samples/servers/mlserver1.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

## Deploy Model

In [ ]:
!cat ../../operator/samples/models/sklearn-iris-gs.yaml

In [ ]:
!kubectl create -f ../../operator/samples/models/sklearn-iris-gs.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!kubectl get server mlserver1 -n seldon-mesh -o jsonpath='{.status}' | jq .

## Deploy Second Server

In [ ]:
!cat ../../operator/samples/servers/mlserver2.yaml

In [ ]:
!kubectl create -f ../../operator/samples/servers/mlserver2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

## Delete First Server

In [ ]:
!kubectl delete -f ../../operator/samples/servers/mlserver1.yaml

In [ ]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!kubectl delete -f ../../operator/samples/models/sklearn-iris-gs.yaml

In [ ]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!kubectl delete -f ../../operator/samples/servers/mlserver2.yaml